In [2]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
import os
import folium

# Import R packages in python
geojsonsf = importr('geojsonsf')
dplyr = importr('dplyr')
sf = importr('sf')
stplanr = importr('stplanr')

# Get the network
G = ox.graph_from_place('Montereale Valcellina, Italy', network_type='all')
edges = ox.graph_to_gdfs(G, nodes=False)

# Filter the major roads
major_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
               'tertiary_link', 'trunk', 'trunk_link', 'residential', 'cycleway',
               'living_street', 'unclassified', 'motorway', 'motorway_link',
               'pedestrian', 'steps', 'track']
edges = edges[edges['highway'].isin(major_roads)]

print(edges.columns)
dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/area.tif'
#dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValc_SRTM.tif'

# Convert geopandas dataframe to geojson
edges_json = edges.to_json()

# Use the geojsonsf package to convert the geojson to an sf object
edges_sf = geojsonsf.geojson_sf(edges_json)

# Assign edges_sf and dem_path to the R environment
ro.globalenv['edges_sf'] = edges_sf
ro.globalenv['dem_path'] = dem_path

# Use the robjects.r method to execute R code
r_script = """
library(dplyr)
library(sf)
library(stplanr)
library(raster)
library(slopes)
library(geodist)
library(geojsonsf)
library(lwgeom)

edges_sf$group = rnet_group(edges_sf)
iow_network_clean = edges_sf %>% filter(group == 1) 
# iow_network_segments = rnet_breakup_vertices(iow_network_clean)

# Import and plot DEM
dem = raster::raster(dem_path)
res(dem)

# Get the CRS of the streets and the DEM
street_crs <- st_crs(iow_network_clean)
dem_crs <- st_crs(dem)

# Convert to EPSG:32632
# iow_network_clean = st_transform(iow_network_clean, crs = 32632)

# Check if they are different
if (street_crs != dem_crs) {
  # Convert to EPSG:32632
  iow_network_clean = st_transform(iow_network_clean, crs = 32632)

  # Define the transformer
  st_crs_4326 <- st_crs(4326)

  # Transform road network bounding box to match the CRS of the DEM
  edges_bounds <- st_bbox(iow_network_clean)
  edges_bounds_transformed <- st_transform(st_as_sfc(edges_bounds, crs = st_crs_4326), crs = dem_crs)
}

network = iow_network_clean

#slope calculus
network$slope = slope_raster(network, dem)
network$slope = network$slope*100 

# Compute the summary
summary_stats = summary(network$slope)
# Print each individual statistic
cat("Minimum: ", summary_stats[1], "\n")
cat("1st Quartile: ", summary_stats[2], "\n")
cat("Median: ", summary_stats[3], "\n")
cat("Mean: ", summary_stats[4], "\n")
cat("3rd Quartile: ", summary_stats[5], "\n")
cat("Maximum: ", summary_stats[6], "\n")
#As we can see in Montereale case, more that the half of the streets are under 2.7 degrees

network$slope_class = network$slope %>%
  cut(
    breaks = c(0, 3, 5, 8, 10, 20, Inf),
    labels = c("0-3: flat", "3-5: mild", "5-8: medium", "8-10: hard", 
               "10-20: extreme", ">20: impossible"),
    right = F
  )
round(prop.table(table(network$slope_class))*100,1)

print(round(prop.table(table(network$slope_class))*100,1))

# Save the sf object to geojson
geojson_file <- tempfile(fileext = ".geojson")
sf::st_write(network, geojson_file)
geojson_file
"""

# Execute the R code to get the path of the geojson file
geojson_file_path = ro.r(r_script)[0]

# Read the GeoJSON file as a GeoDataFrame
edges = gpd.read_file(geojson_file_path)
# print(edges.crs)
# Delete the temporary file
os.remove(geojson_file_path)

# Now you can use network_gdf in Python
#print(network_gdf.head())
print(edges["slope_class"])

print(edges.columns)


R[write to console]: Caricamento del pacchetto richiesto: sp

R[write to console]: The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)



Index(['osmid', 'name', 'highway', 'oneway', 'reversed', 'length', 'ref',
       'geometry', 'lanes', 'bridge', 'maxspeed', 'junction', 'access',
       'est_width', 'tunnel', 'service'],
      dtype='object')


R[write to console]: 
Caricamento pacchetto: ‘raster’


R[write to console]: Il seguente oggetto è mascherato da ‘package:dplyr’:

    select


R[write to console]: Linking to liblwgeom 3.0.0beta1 r16016, GEOS 3.11.0, PROJ 9.1.0



  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~00s           |++                                                | 2 % ~00s           |++                                                | 3 % ~00s           |+++                                               | 4 % ~00s           |+++                                               | 5 % ~00s           |++++                                              | 6 % ~00s           |++++                                              | 7 % ~00s           |+++++                                             | 8 % ~00s           |+++++                                             | 9 % ~00s           |++++++                                            | 10% ~00s           |++++++                                            | 11% ~01s           |+++++++                                           | 12% ~01s           |+++++++                                           | 14% ~01s 

In [3]:
# Reproject to WGS84
edges = edges.to_crs(epsg=4326)

# Create a colormap for slope classes
color_palette = ["#267300", "#70A800", "#FFAA00", "#E60000", "#A80000", "#730000"]
slope_classes = ["0-3: flat", "3-5: mild", "5-8: medium", "8-10: hard", "10-20: extreme", ">20: impossible"]
colors = dict(zip(slope_classes, color_palette))

# Calculate the mean of latitudes and longitudes
mean_latitude = edges.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = edges.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a folium map centered on the mean of latitudes and longitudes
map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11)

# Add slope information to the map
for _, row in edges.iterrows():
    color = colors.get(str(row['slope_class']), "#000000")  # default color is black
    folium.GeoJson(
        row['geometry'], 
        style_function=lambda _, color=color: {'color': color}  # use default argument to capture color
    ).add_to(map_osm)

# Create a custom legend HTML string
legend_html = '''
<div style="position: fixed; top: 10px; right: 10px; z-index: 1000; background-color: white; padding: 5px; border: 1px solid grey; font-size: 12px;">
<p><b>Slope</b></p>
'''
for slope_class, color in colors.items():
    legend_html += f'<p><i class="fa fa-square" style="color:{color};"></i> {slope_class}</p>'
legend_html += '</div>'

# Add the legend HTML to the map
map_osm.get_root().html.add_child(folium.Element(legend_html))

# Save the map
map_osm.save('/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/slope_map.html')

# Display the map 
map_osm